In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
rest = np.loadtxt("rest_data.txt")
right = np.loadtxt("right_data.txt")
left = np.loadtxt("left_data.txt")

rest_reshaped = np.reshape(rest, (rest.shape[0], rest.shape[1] // 2, 2))
right_reshaped = np.reshape(right, (right.shape[0], right.shape[1] // 2, 2))
left_reshaped = np.reshape(left, (left.shape[0], left.shape[1] // 2, 2))

# rest_label = np.zeros((rest.shape[0], 3))
# rest_label[:, 0] = 1
right_label = np.zeros((right.shape[0], 2))
right_label[:, 0] = 1
left_label = np.zeros((left.shape[0], 2))
left_label[:, 1] = 1

In [3]:
print(rest_reshaped.shape)
print(right_reshaped.shape)
print(left_reshaped.shape)

(4768, 640, 2)
(2352, 640, 2)
(2396, 640, 2)


In [4]:
data = np.concatenate((right_reshaped, left_reshaped), axis = 0)
labels = np.concatenate((right_label, left_label), axis = 0)
print(data.shape)
print(labels.shape)
print(np.max(data))
print(np.min(data))

(4748, 640, 2)
(4748, 2)
0.000652
-0.0005909999999999999


In [5]:
randomize = np.arange(0, data.shape[0], 1)
randomize = np.random.permutation(randomize)

data = data[randomize]
labels = labels[randomize]

print(data.shape)
print(labels.shape)

(4748, 640, 2)
(4748, 2)


In [6]:
# 90% training, 10% test
# training_x, training_y = data[0:8564], labels[0:8564]
# testing_x, testing_y = data[8565:], labels[8565:]

training_x, training_y = data[0:4273], labels[0:4273]
testing_x, testing_y = data[4274:], labels[4274:]

In [7]:
normalizer = keras.layers.experimental.preprocessing.Normalization(mean=0., variance=1.)
normalizer.adapt(training_x)

In [8]:
inputs = keras.Input(shape=(640, 2, 1))

In [9]:
from tensorflow.keras import layers

x = normalizer(inputs)

x = layers.Conv2D(filters=25, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.Conv2D(filters=25, kernel_size=[1,2], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=50, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=100, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)
x = layers.SpatialDropout2D(rate=0.5)(x)

x = layers.MaxPooling2D(pool_size=[3,1], strides = [3,1], padding="valid")(x)

x = layers.Conv2D(filters=200, kernel_size=[11,1], strides=[1,1], padding="valid")(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(alpha=0.01)(x)

x = layers.MaxPooling2D(pool_size=[2,1], strides = [2,1], padding="valid")(x)

x = layers.Flatten()(x)

num_classes = 2
outputs = layers.Dense(num_classes, activation="softmax")(x)

In [10]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 640, 2, 1)]       0         
_________________________________________________________________
normalization (Normalization (None, 640, 2, 2)         5         
_________________________________________________________________
conv2d (Conv2D)              (None, 630, 2, 25)        575       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 630, 2, 25)        0         
_________________________________________________________________
spatial_dropout2d (SpatialDr (None, 630, 2, 25)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 630, 1, 25)        1275      
_________________________________________________________________
batch_normalization (BatchNo (None, 630, 1, 25)        100   

In [17]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.BinaryCrossentropy(), metrics=keras.metrics.BinaryAccuracy())

In [18]:
history = model.fit(training_x, training_y,
          batch_size=128, epochs=50)

Epoch 1/50
34/34 [==============================] - 6s 156ms/step - loss: 0.7047 - binary_accuracy: 0.5546
Epoch 2/50
34/34 [==============================] - 5s 158ms/step - loss: 0.6987 - binary_accuracy: 0.5553
Epoch 3/50
34/34 [==============================] - 5s 161ms/step - loss: 0.7070 - binary_accuracy: 0.5364
Epoch 4/50
34/34 [==============================] - 5s 157ms/step - loss: 0.7010 - binary_accuracy: 0.5451
Epoch 5/50
34/34 [==============================] - 5s 153ms/step - loss: 0.6980 - binary_accuracy: 0.5514
Epoch 6/50
34/34 [==============================] - 5s 154ms/step - loss: 0.7002 - binary_accuracy: 0.5560
Epoch 7/50
34/34 [==============================] - 5s 154ms/step - loss: 0.7011 - binary_accuracy: 0.5479
Epoch 8/50
34/34 [==============================] - 6s 165ms/step - loss: 0.6997 - binary_accuracy: 0.5488
Epoch 9/50
34/34 [==============================] - 5s 156ms/step - loss: 0.6928 - binary_accuracy: 0.5699
Epoch 10/50
34/34 [==================

KeyboardInterrupt: 